# Entrenamiento de Modelo con AdaGrad (Adaptive Gradient)

Este notebook implementa un modelo de regresión usando **AdaGrad (Adaptive Gradient Algorithm)** para predecir valores basados en datos de series temporales del archivo `dryer.dat`.

## ¿Qué es AdaGrad?

AdaGrad es un algoritmo de optimización adaptativo que **ajusta el learning rate de forma individual para cada parámetro**. Los parámetros que reciben gradientes grandes tienen su learning rate reducido, mientras que los parámetros con gradientes pequeños mantienen un learning rate mayor.

### Características clave:
- **Learning rates adaptativos**: Cada peso tiene su propio learning rate
- **Ideal para datos esparsos**: Funciona muy bien en NLP y datos categóricos
- **Sin necesidad de ajuste manual**: El learning rate se adapta automáticamente
- **Acumulación de gradientes**: Mantiene un historial de gradientes al cuadrado

## División de datos:
- **70%** - Entrenamiento (Train)
- **20%** - Validación (Valid)
- **10%** - Prueba (Test)

**Importante**: Los datos se dividen de forma secuencial para respetar el orden temporal.

## 1. Importar Librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

## 2. Cargar y Explorar los Datos

In [2]:
# Cargar datos desde dryer.dat
print("Cargando datos desde dryer.dat...")
data = np.loadtxt('dryer.dat')

# Separar características (X) y variable objetivo (y)
X = data[:, 0].reshape(-1, 1)  # Primera columna como entrada
y = data[:, 1].reshape(-1, 1)  # Segunda columna como salida

print(f"Datos cargados: {len(X)} muestras")
print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")

Cargando datos desde dryer.dat...
Datos cargados: 1000 muestras
Forma de X: (1000, 1)
Forma de y: (1000, 1)


## 3. División Secuencial de los Datos

Como son datos de series temporales, NO mezclamos los datos. Mantenemos el orden temporal.

In [3]:
# División de datos SECUENCIAL (para series temporales): 70% train, 20% valid, 10% test
# No mezclamos los datos, mantenemos el orden temporal
n_samples = len(X)
train_end = int(0.7 * n_samples)
valid_end = int(0.9 * n_samples)

# División secuencial
X_train = X[:train_end]
y_train = y[:train_end]

X_valid = X[train_end:valid_end]
y_valid = y[train_end:valid_end]

X_test = X[valid_end:]
y_test = y[valid_end:]

print(f"\nDivisión de datos SECUENCIAL (series temporales):")
print(f"Train: {len(X_train)} muestras ({len(X_train)/len(X)*100:.1f}%) - Índices [0:{train_end}]")
print(f"Valid: {len(X_valid)} muestras ({len(X_valid)/len(X)*100:.1f}%) - Índices [{train_end}:{valid_end}]")
print(f"Test:  {len(X_test)} muestras ({len(X_test)/len(X)*100:.1f}%) - Índices [{valid_end}:{n_samples}]")


División de datos SECUENCIAL (series temporales):
Train: 700 muestras (70.0%) - Índices [0:700]
Valid: 200 muestras (20.0%) - Índices [700:900]
Test:  100 muestras (10.0%) - Índices [900:1000]


## 4. Normalización/Estandarización de los Datos

In [4]:
# Normalizar datos usando StandardScaler
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_norm = scaler_X.fit_transform(X_train)
X_valid_norm = scaler_X.transform(X_valid)
X_test_norm = scaler_X.transform(X_test)

y_train_norm = scaler_y.fit_transform(y_train)
y_valid_norm = scaler_y.transform(y_valid)
y_test_norm = scaler_y.transform(y_test)

print("Datos normalizados exitosamente")

Datos normalizados exitosamente


## 5. Implementación del Modelo con AdaGrad

Implementamos desde cero un modelo de regresión usando **AdaGrad (Adaptive Gradient Algorithm)**.

### Fórmula de AdaGrad:
```
G_t = G_{t-1} + (∇L)²           # Acumular gradientes al cuadrado
θ_t = θ_{t-1} - (α / √(G_t + ε)) * ∇L
```

Donde:
- `G_t` es la suma acumulada de gradientes al cuadrado
- `α` es el learning rate inicial
- `ε` es un término pequeño para evitar división por cero (típicamente 1e-8)
- Cada parámetro tiene su propio `G_t`

In [5]:
# Implementación de AdaGrad (Adaptive Gradient Algorithm)
class AdaGradRegressor:
    def __init__(self, learning_rate=0.1, n_epochs=700, batch_size=32, epsilon=1e-8):
        self.learning_rate = learning_rate  # Learning rate inicial (típicamente más alto que SGD)
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.epsilon = epsilon  # Término pequeño para estabilidad numérica
        self.weights = None
        self.bias = None
        self.G_w = None  # Acumulador de gradientes al cuadrado para pesos
        self.G_b = None  # Acumulador de gradientes al cuadrado para bias
        self.train_losses = []
        self.valid_losses = []
        
    def _initialize_parameters(self, n_features):
        """Inicializar pesos, bias y acumuladores"""
        self.weights = np.random.randn(n_features, 1) * 0.01
        self.bias = np.zeros((1, 1))
        # CLAVE DE ADAGRAD: Inicializar acumuladores en cero
        self.G_w = np.zeros((n_features, 1))
        self.G_b = np.zeros((1, 1))
    
    def _compute_loss(self, y_true, y_pred):
        """Calcular MSE (Mean Squared Error)"""
        return np.mean((y_true - y_pred) ** 2)
    
    def _forward(self, X):
        """Propagación hacia adelante"""
        return np.dot(X, self.weights) + self.bias
    
    def _backward(self, X, y_true, y_pred):
        """Propagación hacia atrás (calcular gradientes)"""
        n_samples = X.shape[0]
        
        # Gradientes
        dw = (-2/n_samples) * np.dot(X.T, (y_true - y_pred))
        db = (-2/n_samples) * np.sum(y_true - y_pred)
        
        return dw, db
    
    def fit(self, X_train, y_train, X_valid=None, y_valid=None):
        """Entrenar el modelo usando AdaGrad"""
        n_samples, n_features = X_train.shape
        self._initialize_parameters(n_features)
        
        # Entrenar por épocas
        for epoch in range(self.n_epochs):
            X = X_train
            Y = y_train
            
            # Mini-batch AdaGrad
            for i in range(0, n_samples, self.batch_size):
                X_batch = X[i:i+self.batch_size]
                y_batch = Y[i:i+self.batch_size]
                
                # ===== PASO 1: Forward pass =====
                y_pred = self._forward(X_batch)
                
                # ===== PASO 2: Backward pass (calcular gradientes) =====
                dw, db = self._backward(X_batch, y_batch, y_pred)
                
                # ===== PASO 3: Acumular gradientes al cuadrado (CLAVE DE ADAGRAD) =====
                # G_t = G_{t-1} + (∇L)²
                self.G_w += dw ** 2  # Elemento por elemento
                self.G_b += db ** 2
                
                # ===== PASO 4: Actualizar parámetros con learning rate adaptativo =====
                # θ = θ - (α / √(G + ε)) * ∇L
                # El denominador √(G + ε) adapta el learning rate para cada parámetro
                
                # Learning rate adaptativo para cada peso
                adapted_lr_w = self.learning_rate / (np.sqrt(self.G_w + self.epsilon))
                adapted_lr_b = self.learning_rate / (np.sqrt(self.G_b + self.epsilon))
                
                # Actualizar usando learning rates adaptativos
                self.weights -= adapted_lr_w * dw
                self.bias -= adapted_lr_b * db
            
            # Calcular pérdida en train
            y_train_pred = self._forward(X_train)
            train_loss = self._compute_loss(y_train, y_train_pred)
            self.train_losses.append(train_loss)
            
            # Calcular pérdida en validación (si se proporciona)
            if X_valid is not None and y_valid is not None:
                y_valid_pred = self._forward(X_valid)
                valid_loss = self._compute_loss(y_valid, y_valid_pred)
                self.valid_losses.append(valid_loss)
                
                if (epoch + 1) % 10 == 0:
                    print(f"Época {epoch+1}/{self.n_epochs} - Train Loss: {train_loss:.6f} - Valid Loss: {valid_loss:.6f}")
            else:
                if (epoch + 1) % 10 == 0:
                    print(f"Época {epoch+1}/{self.n_epochs} - Train Loss: {train_loss:.6f}")
    
    def predict(self, X):
        """Hacer predicciones"""
        return self._forward(X)
    
    def evaluate(self, X, y):
        """Evaluar el modelo"""
        y_pred = self.predict(X)
        mse = self._compute_loss(y, y_pred)
        rmse = np.sqrt(mse)
        
        # R² score
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        ss_res = np.sum((y - y_pred) ** 2)
        r2 = 1 - (ss_res / ss_tot)
        
        return {'MSE': mse, 'RMSE': rmse, 'R2': r2}
    
    def get_effective_learning_rates(self):
        """Obtener los learning rates efectivos actuales para cada parámetro"""
        lr_w = self.learning_rate / (np.sqrt(self.G_w + self.epsilon))
        lr_b = self.learning_rate / (np.sqrt(self.G_b + self.epsilon))
        return lr_w, lr_b

print("Clase AdaGradRegressor definida exitosamente")

Clase AdaGradRegressor definida exitosamente


## 6. Entrenar el Modelo con AdaGrad

### Hiperparámetros:
- **learning_rate**: 0.1 (más alto que SGD porque se adapta automáticamente)
- **n_epochs**: 200 (número de épocas)
- **batch_size**: 32 (tamaño del mini-batch)
- **epsilon**: 1e-8 (para estabilidad numérica)

In [6]:
# Crear y entrenar el modelo con AdaGrad
print("="*60)
print("ENTRENAMIENTO DEL MODELO ADAGRAD")
print("="*60)

model = AdaGradRegressor(learning_rate=0.1, n_epochs=200, batch_size=32, epsilon=1e-8)
model.fit(X_train_norm, y_train_norm, X_valid_norm, y_valid_norm)

ENTRENAMIENTO DEL MODELO ADAGRAD
Época 10/200 - Train Loss: 0.971138 - Valid Loss: 0.861766
Época 20/200 - Train Loss: 0.971134 - Valid Loss: 0.861813
Época 30/200 - Train Loss: 0.971120 - Valid Loss: 0.862097
Época 40/200 - Train Loss: 0.971111 - Valid Loss: 0.862301
Época 50/200 - Train Loss: 0.971106 - Valid Loss: 0.862447
Época 60/200 - Train Loss: 0.971101 - Valid Loss: 0.862558
Época 70/200 - Train Loss: 0.971098 - Valid Loss: 0.862644
Época 80/200 - Train Loss: 0.971096 - Valid Loss: 0.862715
Época 90/200 - Train Loss: 0.971093 - Valid Loss: 0.862774
Época 100/200 - Train Loss: 0.971092 - Valid Loss: 0.862824
Época 110/200 - Train Loss: 0.971090 - Valid Loss: 0.862867
Época 120/200 - Train Loss: 0.971089 - Valid Loss: 0.862904
Época 130/200 - Train Loss: 0.971088 - Valid Loss: 0.862938
Época 140/200 - Train Loss: 0.971087 - Valid Loss: 0.862968
Época 150/200 - Train Loss: 0.971086 - Valid Loss: 0.862994
Época 160/200 - Train Loss: 0.971085 - Valid Loss: 0.863019
Época 170/200 - 

## 7. Evaluación del Modelo

### 7.1 Evaluación en Train

In [7]:
# Evaluar en conjunto de entrenamiento
print("\n" + "="*60)
print("EVALUACIÓN EN CONJUNTO DE ENTRENAMIENTO")
print("="*60)
train_metrics = model.evaluate(X_train_norm, y_train_norm)
for metric, value in train_metrics.items():
    print(f"{metric}: {value:.6f}")


EVALUACIÓN EN CONJUNTO DE ENTRENAMIENTO
MSE: 0.971083
RMSE: 0.985435
R2: 0.028917


### 7.2 Evaluación en Validación

In [8]:
# Evaluar en conjunto de validación
print("\n" + "="*60)
print("EVALUACIÓN EN CONJUNTO DE VALIDACIÓN")
print("="*60)
valid_metrics = model.evaluate(X_valid_norm, y_valid_norm)
for metric, value in valid_metrics.items():
    print(f"{metric}: {value:.6f}")


EVALUACIÓN EN CONJUNTO DE VALIDACIÓN
MSE: 0.863097
RMSE: 0.929030
R2: -0.065691


### 7.3 Evaluación en Test

In [9]:
# Evaluar en conjunto de prueba
print("\n" + "="*60)
print("EVALUACIÓN EN CONJUNTO DE PRUEBA (TEST)")
print("="*60)
test_metrics = model.evaluate(X_test_norm, y_test_norm)
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.6f}")


EVALUACIÓN EN CONJUNTO DE PRUEBA (TEST)
MSE: 0.747110
RMSE: 0.864356
R2: -0.053222
